## Install libraries

In [ ]:
!pip install gymnasium
!pip install tqdm

## Import libraries

In [2]:
import gymnasium as gym
import numpy as np
from tqdm import tqdm

## Create Taxi-v3 environment

In [46]:
env = gym.make("Taxi-v3", render_mode='ansi')

## Set environment to illustration's state

In [47]:
state = env.unwrapped.encode(3, 1, 2, 0)
# (taxi row, taxi column, passenger index, destination index)

print("State:", state)
env.s = state

# default reward values assigned to the illustration’s state
# {action: [(probability, next_state, reward, done)]}
env.unwrapped.P[328]

State: 328


{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

## Initialize Q-table with zeros

In [48]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

## Training Q-learning model

In [49]:
# Parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate

# Function to choose an action
def choose_action(state):
    if np.random.uniform(0, 1) < epsilon:
        return env.action_space.sample() # Explore action space
    else:
        return np.argmax(q_table[state]) # Exploit learned values

# Training the Q-learning algorithm
num_episodes = 10000
for episode in tqdm(range(num_episodes)):
    state, info = env.reset()
    done = False

    while not done:
        action = choose_action(state)
        next_state, reward, done, truncated, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        state = next_state

# print Q-values at the illustration’s state
q_table[328]

100%|██████████| 10000/10000 [00:10<00:00, 950.78it/s]


array([-3.30986883,  9.60071394, -3.07910832, -2.84574211, -5.93685967,
       -4.39997893])

## Evaluation

In [52]:
num_episodes = 100  # Number of episodes for evaluation
total_penalties = 0
total_timesteps = 0
total_rewards = 0

# Run episodes for evaluation
for episode in tqdm(range(num_episodes)):
    state, info = env.reset()
    penalties = 0
    episode_reward = 0

    done = False
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, truncated, info = env.step(action)

        episode_reward += reward
        total_timesteps += 1

        if reward == -10:
            penalties += 1

    total_penalties += penalties
    total_rewards += episode_reward

# Calculate metrics
avg_penalties_per_episode = total_penalties / num_episodes
avg_timesteps_per_episode = total_timesteps / num_episodes
avg_rewards_per_move = total_rewards / total_timesteps

print("Average number of penalties per episode:", avg_penalties_per_episode)
print("Average number of timesteps per episode:", avg_timesteps_per_episode)
print("Average rewards per move:", avg_rewards_per_move)

100%|██████████| 100/100 [00:00<00:00, 2151.09it/s]

Average number of penalties per episode: 0.0
Average number of timesteps per episode: 13.02
Average rewards per move: 0.6129032258064516
